In [53]:
import pandas as pd

In [54]:
xls = pd.ExcelFile("Assignment_3_Data.xlsx")
df1 = pd.read_excel(xls, 'Firm Characteristics',skiprows=6,header=1) # Corporate/Financial Characteristics
df2 = pd.read_excel(xls, 'Aggregates',skiprows=7,header=1) # Aggregate summary data
df3 = pd.read_excel(xls, 'Stock Prices',header=1) # This is the time series data


In [55]:
df3.dropna(axis="columns", inplace=True)
df3 = df3.apply(pd.to_numeric, errors='coerce').dropna(axis="columns")

In [56]:
sample = df3.sample(n=50, axis=1)
sample

,IMGN,CMPR,Y,CTAS,OMI,ASMB,CBI,PACW,TRMB,SBSI,...,LNN,OLBK,WEYS,PEGA,VMC,TJX,ORBC,GIII,NKTR,EXC
0,11.54,30.51,284.86,32.59268,23.70040,39.95,37.35839,16.17505,20.815,13.51760,...,52.33555,7.53174,21.49945,14.41534,38.78268,29.71977,3.18,12.405,5.76,32.90290
1,11.38,30.70,285.00,33.67016,23.50627,40.50,37.45499,16.42329,20.105,13.28943,...,51.94205,7.53174,21.64300,13.75610,38.41127,30.05564,3.18,12.525,5.68,32.37107
2,11.82,31.03,285.92,33.76727,23.42187,40.60,37.00094,16.69554,20.385,13.24010,...,52.46672,7.48525,21.65145,13.65843,38.72404,30.93263,3.15,12.095,5.76,32.26157
3,11.77,31.35,286.45,34.50717,23.58224,40.25,37.15551,16.55141,20.075,13.26476,...,51.74530,7.45736,21.49945,13.20429,38.40150,31.00260,3.20,11.690,5.73,32.13644
4,11.89,30.45,284.95,34.58116,23.28683,40.00,37.58058,16.76761,20.870,13.32643,...,51.92331,7.22490,21.55856,13.34591,38.60675,30.64808,3.19,11.475,5.76,32.37889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1461,5.49,108.27,565.04,151.49000,28.59000,30.38,15.76000,48.37000,40.780,36.54000,...,91.10000,29.35000,27.83000,59.15000,121.08000,72.20000,11.15,26.550,24.55,39.74000
1462,5.47,107.75,563.32,150.85000,29.02000,29.74,16.56000,48.68000,41.120,36.70000,...,92.36000,29.40000,27.61000,58.75000,123.76000,71.74000,11.09,26.640,24.14,39.96000
1463,5.39,107.10,565.67,150.16000,28.60000,30.56,16.24000,47.89000,41.200,36.79000,...,91.96000,29.71000,27.77000,57.70000,122.33000,71.44000,10.96,26.640,24.17,40.07000
1464,5.19,108.00,571.44,151.55000,27.66000,30.97,16.71000,48.88000,41.210,37.00000,...,92.51000,30.30000,28.18000,57.60000,123.01000,71.47000,11.24,26.170,23.66,39.88000


In [57]:
returns = df3.reset_index(drop=True).drop(["Date"], axis=1).astype(float).pct_change()
mean_returns = returns.mean()
cov_matrix = returns.cov()